Source: https://www.datascience.net/fr/challenge/25/details

# Le Meilleur Data Scientist de France

La première édition de ce challenge éclair, organisé par FrenchData, a réuni au cours d’une soirée 250 passionnés de data. Ce challenge est désormais ouvert à tous. Le thème était : combien coûte une boite de médicament ? 

L’objectif de ce challenge est de mettre au point un modèle prédictif pour estimer combien coûte une boite de médicament, afin par exemple d’aider à en fixer le prix, ou bien détecter des anomalies dans les tarifs.

## Contexte

En France, les prix des médicaments sont pour la plupart administrés, même si des prix libres existent pour certains d'entre eux. Le processus de fixation des prix suit un cadre réglementaire bien précis [1] [2].

Pourtant, des écarts de prix peuvent subsister [3].

L'objectif de ce challenge est de mettre au point un modèle prédictif pour estimer combien coûte une boite de médicament, afin par exemple d'aider à en fixer le prix, ou bien détecter des anomalies dans les tarifs. 

## Données

Un même médicament peut se commercialiser sous différentes formes (taille de la boite, type de gélule, ...) faisant varier le prix de la boîte.

Le fichier principal à utiliser pour le concours est boites_medicaments.csv. 

Il est encodé en UTF-8, le séparateur est « ; ». 

Il contient les informations de chaque boîte de médicament, et leur prix :

| Champ             |      Description                                                                                                                                                           |
|-------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| libelle           |      Description du contenu de la boîte de médicament (exemple : 5 ampoules en verre de 5 ml)                                                                              |
| libelle_xxx       |      13 variables qui codent la présence de certains mots dans le libellé (exemple libelle_ampoule = 1 si le mot « ampoule » apparait dans le champs « libelle », 0 sinon. |
| nb_xxx            |      14 variables qui indiquent certaines quantités présentes dans les libellés (exemple : nb_ampoule = 32 si « 32 ampoules » apparaît dans le champ « libelle », sinon 0  |
| statut            |      Statut administratif de la boîte de médicament                                                                                                                        |
| etat commerc      |      Etat de commercialisation de la boîte de médicament tel que déclaré par le titulaire de l'AMM                                                                         |
| agrement col      |      Agrément aux collectivités ("oui", "non" ou « inconnu »)                                                                                                              |
| tx rembours       |      Taux de remboursement                                                                                                                                                 |
| forme  pharma     |      Forme pharmaceutique (comprimé pelliculé, comprimé sécable, ...)                                                                                                      |
| voies  admin      |      Voies d'administration (avec un séparateur « , » entre chaque valeur quand il y en a plusieurs : orale, ophtalmique, intramusculaire, ...)                            |
| statut admin      |      Statut administratif de l’autorisation de mise sur le marché (AMM) : active, abrogée, ...                                                                             |
| date déclar année |       Année de la déclaration de commercialisation de la boîte de médicament                                                                                               |
| date amm année    |       Année d’Autorisation de Mise sur le Marché du médicament                                                                                                             |
| type proc         |       Type de procédure d'autorisation de mise sur le marché (AMM) (Nationale, décentralisée, reconnaissance mutuelle, ...)                                                |
| titulaires        |      Laboratoire titulaire du médicament. S'il y a plusieurs titulaires, ils sont séparés par des « , »                                                                    |
| substances        |      Substances présentes dans le médicament. S'il y en a plusieurs, elles sont séparées par des « , »                                                                     |
| prix              |      Le prix de la boîte de médicament (présent dans le fichier « train » uniquement, à prédire dans le fichier « test »)                                                  |


## Critère de performance 

La métrique utilisée pour mesurer la précision du modèle est Mean Absolute Percentage Error (MAPE) : 
https://en.wikipedia.org/wiki/Mean_absolute_percentage_error

### Format du fichier à fournir

Le fichier que doit fournir le candidat est un fichier au format .csv, dont la structure est la suivante :

| Id; | Prix    |
|-----|---------|
| 1;  | 2.3987  |
| 2;  | 7.3873  |
| ... | ...     |


# Découverte des données

Les données sont stockées sur HDFS dans `/gsemet/meilleur_data_scientist`

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale = 1.5)
pd.set_option('display.max_columns', 500)

train = pd.read_csv('/mnt/hdfs/gsemet/meilleur_data_scientist/boites_medicaments_train.csv', sep=';')
test  = pd.read_csv('/mnt/hdfs/gsemet/meilleur_data_scientist/boites_medicaments_test.csv', sep=';')

In [4]:
train.head()

,libelle,libelle_plaquette,libelle_ampoule,libelle_flacon,libelle_tube,libelle_stylo,libelle_seringue,libelle_pilulier,libelle_sachet,libelle_comprime,libelle_gelule,libelle_film,libelle_poche,libelle_capsule,nb_plaquette,nb_ampoule,nb_flacon,nb_tube,nb_stylo,nb_seringue,nb_pilulier,nb_sachet,nb_comprime,nb_gelule,nb_film,nb_poche,nb_capsule,nb_ml,statut,etat commerc,agrement col,tx rembours,forme pharma,voies admin,statut admin,date declar annee,date amm annee,type proc,titulaires,substances,prix
0,3 plaquette(s) thermoformée(s) PVC-Aluminium d...,1,0,0,0,0,0,0,0,1,0,0,0,0,3,0,0,0,0,0,0,0,28,0,0,0,0,0.0,Présentation active,Déclaration de commercialisation,oui,65%,comprimé pelliculé,orale,Autorisation active,2014,2014,Procédure décentralisée,MAJORELLE LUXEMBOURG SOPARFI (LUXEMBOURG),DÉSOGESTREL,2.83
1,plaquette(s) thermoformée(s) aluminium de 28 c...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,28,0,0,0,0,0.0,Présentation active,Déclaration de commercialisation,oui,65%,comprimé à croquer,orale,Autorisation active,2013,2009,Procédure de reconnaissance mutuelle,TEVA SANTE,"MONTÉLUKAST ACIDE, MONTÉLUKAST SODIQUE",14.30
2,plaquette(s) thermoformée(s) PVC-aluminium de ...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0.0,Présentation active,Déclaration de commercialisation,oui,65%,gélule à libération prolongée,orale,Autorisation active,2000,1996,Procédure nationale,MYLAN SAS,CHLORHYDRATE DE DILTIAZEM,5.66
3,30 sachet(s)-dose(s) papier aluminium complexe...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0.0,Présentation active,Déclaration de commercialisation,oui,65%,granulés à libération prolongée,orale,Autorisation active,2005,2004,Procédure nationale,SANOFI AVENTIS FRANCE,"ACIDE VALPROÏQUE, VALPROATE DE SODIUM",24.27
4,plaquette(s) thermoformée(s) PVC-Aluminium de ...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0.0,Présentation active,Déclaration d'arrêt de commercialisation,non,100%,comprimé pelliculé,orale,Autorisation active,2015,2010,Procédure nationale,TEVA SANTE,LÉTROZOLE,59.94


In [5]:
test.head()

,id,libelle,libelle_plaquette,libelle_ampoule,libelle_flacon,libelle_tube,libelle_stylo,libelle_seringue,libelle_pilulier,libelle_sachet,libelle_comprime,libelle_gelule,libelle_film,libelle_poche,libelle_capsule,nb_plaquette,nb_ampoule,nb_flacon,nb_tube,nb_stylo,nb_seringue,nb_pilulier,nb_sachet,nb_comprime,nb_gelule,nb_film,nb_poche,nb_capsule,nb_ml,statut,etat commerc,agrement col,tx rembours,forme pharma,voies admin,statut admin,date declar annee,date amm annee,type proc,titulaires,substances
0,1,plaquette(s) thermoformée(s) polyamide alumini...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0,0,0,0.0,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé gastro-résistant(e),orale,Autorisation active,2013,2008,Procédure de reconnaissance mutuelle,TEVA SANTE,"PANTOPRAZOLE SODIQUE SESQUIHYDRATÉ, PANTOPRAZOLE"
1,2,plaquette(s) thermoformée(s) PVC PVDC aluminiu...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,0.0,Présentation active,Déclaration de commercialisation,non,30%,capsule,vaginale,Autorisation active,2012,1997,Procédure nationale,INNOTECH INTERNATIONAL,"NYSTATINE, SULFATE DE POLYMYXINE B, SULFATE DE..."
2,3,plaquette(s) thermoformée(s) PVC-Aluminium de ...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,84,0,0,0,0,0.0,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,comprimé,orale,Autorisation abrogée,2013,2006,Procédure nationale,ACTAVIS FRANCE,PRAVASTATINE SODIQUE
3,4,5 cartouche(s) en verre de 3 ml dans stylo pré...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,Présentation active,Déclaration d'arrêt de commercialisation,non,65%,suspension injectable,sous-cutanée,Autorisation active,2012,2000,Procédure centralisée,SANOFI AVENTIS (ALLEMAGNE),INSULINE HUMAINE RECOMBINANTE ISOPHANE BIPHASI...
4,5,5 poche(s) à 3 compartiments polyamide polypro...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5,0,1875.0,Présentation active,Déclaration de commercialisation,oui,65%,émulsion pour perfusion,intraveineuse,Autorisation active,2011,2011,Procédure décentralisée,B BRAUN MELSUNGEN,"PHOSPHATE MONOSODIQUE DIHYDRATÉ, ARGININE, MAG..."
